In [1]:
data_path = "/Users/connorkitchings/Library/CloudStorage/GoogleDrive-connor.kitchings@gmail.com/My Drive/Colab Notebooks/CFB Notebooks/Data/"
module_path = "/Users/connorkitchings/Desktop/Repositories/CFB-Model/Module/"
logo_path = "/Users/connorkitchings/Desktop/Repositories/CFB-Model/Data/Logos/"

In [2]:
# pip install -r requirements.txt

In [3]:
from typing import Dict, List, Tuple
import pandas as pd
import numpy as np
import xgboost
import sklearn
import os
import time
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from PIL import ImageOps
from tqdm import tqdm as tqdm_
import requests
import ast

# Lists and Dictionaries

In [4]:
FBS_list = [
    "SEC",
    "American Athletic",
    "FBS Independents",
    "Big Ten",
    "Conference USA",
    "Big 12",
    "Mid-American",
    "ACC",
    "Sun Belt",
    "Pac-12",
    "Mountain West",
]
P5schoolsandconferences = {
    "Alabama": "SEC",
    "Arkansas": "SEC",
    "Auburn": "SEC",
    "Florida": "SEC",
    "Georgia": "SEC",
    "Kentucky": "SEC",
    "LSU": "SEC",
    "Mississippi State": "SEC",
    "Missouri": "SEC",
    "Ole Miss": "SEC",
    "South Carolina": "SEC",
    "Tennessee": "SEC",
    "Texas A&M": "SEC",
    "Vanderbilt": "SEC",
    "Illinois": "Big Ten",
    "Indiana": "Big Ten",
    "Iowa": "Big Ten",
    "Maryland": "Big Ten",
    "Michigan": "Big Ten",
    "Michigan State": "Big Ten",
    "Minnesota": "Big Ten",
    "Nebraska": "Big Ten",
    "Northwestern": "Big Ten",
    "Ohio State": "Big Ten",
    "Penn State": "Big Ten",
    "Purdue": "Big Ten",
    "Rutgers": "Big Ten",
    "Wisconsin": "Big Ten",
    "Illinois": "Big Ten",
    "Indiana": "Big Ten",
    "Iowa": "Big Ten",
    "Maryland": "Big Ten",
    "Michigan": "Big Ten",
    "Michigan State": "Big Ten",
    "Minnesota": "Big Ten",
    "Nebraska": "Big Ten",
    "Northwestern": "Big Ten",
    "Ohio State": "Big Ten",
    "Penn State": "Big Ten",
    "Purdue": "Big Ten",
    "Rutgers": "Big Ten",
    "Wisconsin": "Big Ten",
    "Baylor": "Big 12",
    "Iowa State": "Big 12",
    "Kansas": "Big 12",
    "Kansas State": "Big 12",
    "Oklahoma": "Big 12",
    "Oklahoma State": "Big 12",
    "TCU": "Big 12",
    "Texas": "Big 12",
    "Texas Tech": "Big 12",
    "West Virginia": "Big 12",
    "Arizona": "Pac-12",
    "Arizona State": "Pac-12",
    "California": "Pac-12",
    "Colorado": "Pac-12",
    "Oregon": "Pac-12",
    "Oregon State": "Pac-12",
    "Stanford": "Pac-12",
    "UCLA": "Pac-12",
    "USC": "Pac-12",
    "Utah": "Pac-12",
    "Washington": "Pac-12",
    "Washington State": "Pac-12",
    "BYU": "Big 12",
    "Notre Dame": "Independent",
}

In [5]:
stats_dictionary = {
    "ypp": "Yards Per Play",
    "success_rate": "Success Rate",
    "explosive_rate": "Explosive Rate",
    "avg_1st_down_yardage": "Average First Down Yardage",
    "avg_2nd_down_yardage": "Average Second Down Yardage",
    "avg_3rd_down_yardage": "Average Third Down Yardage",
    "3rd_down_conversion_rate": "Third Down Conversion Rate",
    "r_ypp": "Rushing Yards Per Play",
    "r_success_rate": "Rushing Success Rate",
    "opportunity_rate": "Opportunity Rate",
    "stuff_rate": "Stuff Rate",
    "p_ypp": "Passing Yards Per Play",
    "p_success_rate": "Passing Success Rate",
    "sack_rate": "Sack Rate",
    "ptsperdrive": "Points Per Drive",
    "availableyard_rate": "Available Yard Rate",
    "TD_rate": "Touchdown Rate",
    "valuedrive_rate": "Value Drive Rate",
    "eckel_rate": "Eckel Rate",
    "firstdowndrive_rate": "First Down Drive Rate",
    "busteddrive_rate": "Busted Drive Rate",
    "giveaway_rate": "Giveaway Rate",
    "scoredrive_rate": "Score Drive Rate",
    "success_yardspersuccess": "Success Yards Per Success",
    "pts_scoredpersuccess": "Points Scored Per Success",
}

In [ ]:
columns_dictionary = {
    "ypp": "Yards Per Play",
    "success_rate": "Success Rate",
    "explosive_rate": "Explosive Rate",
    "TFL_rate": "Tackle For Loss Rate",
    "turnover_rate": "Turnover Rate",
    "third_down_conversion_rate": "Third Down Conversion Rate",
    "fourth_down_conversion_rate": "Fourth Down Conversion Rate",
    "rushpass_split": "Rush/Pass Split",
    "r_ypp": "Rushing Yards Per Play",
    "r_success_rate": "Rushing Success Rate",
    "r_explosive_rate": "Rushing Explosive Rate",
    "r_successyards_per_success": "Rushing Success Yards Per Success",
    "r_explosiveyards_per_explosive": "Rushing Explosive Yards Per Explosive",
    "stuff_rate": "Stuff Rate",
    "p_ypp": "Passing Yards Per Play",
    "p_success_rate": "Passing Success Rate",
    "p_explosive_rate": "Passing Explosive Rate",
    "p_successyards_per_success": "Passing Success Yards Per Success",
    "p_explosiveyards_per_explosive": "Passing Explosive Yards Per Explosive",
    "completion_rate": "Completion Rate",
    "sack_rate": "Sack Rate",
    "havoc_rate": "Havoc Rate",
    "avg_available_yards": "Average Available Yards",
    "avg_available_yards_gained": "Average Available Yards Gained",
    "redzone_drive_rate": "Redzone Drive Rate",
    "eckel_drive_rate": "Eckel Drive Rate",
    "drive_success_rate": "Drive Success Rate",
    "scoring_drive_rate": "Scoring Drive Rate",
    "pointsscored_per_drive": "Points Scored Per Drive",
    "TD_drive_rate": "Touchdown Drive Rate",
    "FG_drive_rate": "Field Goal Drive Rate",
    "firstdown_drive_rate": "First Down Drive Rate",
    "busted_drive_rate": "Busted Drive Rate",
    "longdrive_rate": "Long Drive Rate",
    "ppa_per_drive": "Predicted Points Added Per Drive",
    "giveaway_drive_rate": "Giveaway Drive Rate",
    "o_penalties_per_drive": "Offensive Penalties Per Drive",
    "o_penaltyyards_per_drive": "Offensive Penalty Yards Per Drive",
    "d_penalties_per_drive": "Defensive Penalties Per Drive",
    "d_penaltyyards_per_drive": "Defensive Penalty Yards Per Drive",
}

# Need to add 1st 2nd and 3rd down yardage, opportunity rate (maybe), pts per drive, and pts per success

# Stat Aggregations

## By Game - Plays Cumulative

In [6]:
def byplay_to_gameplayscum(processed_plays, verbose=False):
    start = time.time()

    # Constants
    FOURTHDOWNGIVEUP_LIST = [
        "Punt",
        "Blocked Punt",
        "Punt Return Touchdown",
        "Blocked Punt Touchdown",
        "Field Goal Good",
        "Field Goal Missed",
        "Blocked Field Goal",
        "Missed Field Goal Return",
        "Missed Field Goal Return Touchdown",
        "Blocked Field Goal Touchdown",
    ]

    # Input validation
    if not isinstance(processed_plays, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")

    # Filter in offensive plays only
    plays = processed_plays[
        (processed_plays["penalty"] == 0)
        & (processed_plays["st"] == 0)
        & (~processed_plays["play_type"].isin(FOURTHDOWNGIVEUP_LIST))
        & (
            ~processed_plays["play_type"].isin(
                ["End of Game", "End of Half", "End of Quarter"]
            )
        )
        # &
        # (processed_plays['garbage'] == 0)
    ].copy()

    # Create down and distance columns
    plays["first_down"] = (plays["down"] == 1).astype(int)
    plays["second_down"] = (plays["down"] == 1).astype(int)
    plays["third_down"] = (plays["down"] == 1).astype(int)
    plays["fourth_down"] = (plays["down"] == 1).astype(int)

    # Create rush and pass success/explosive and success/explosive yards
    plays["rush_success"] = (
        (plays["rush_attempt"] == 1) & (plays["success"] == 1)
    ).astype(int)
    plays["pass_success"] = ((plays["dropback"] == 1) & (plays["success"] == 1)).astype(
        int
    )
    plays["rush_explosive"] = (
        (plays["rush_attempt"] == 1) & (plays["explosive"] == 1)
    ).astype(int)
    plays["pass_explosive"] = (
        (plays["dropback"] == 1) & (plays["explosive"] == 1)
    ).astype(int)
    plays["rush_yards"] = np.where(plays["rush_attempt"] == 1, plays["yards_gained"], 0)
    plays["pass_yards"] = np.where(plays["dropback"] == 1, plays["yards_gained"], 0)
    plays["rush_success_yards"] = np.where(
        plays["rush_success"] == 1, plays["rush_yards"], 0
    )
    plays["rush_explosive_yards"] = np.where(
        plays["rush_explosive"] == 1, plays["rush_yards"], 0
    )
    plays["pass_success_yards"] = np.where(
        plays["pass_success"] == 1, plays["pass_yards"], 0
    )
    plays["pass_explosive_yards"] = np.where(
        plays["pass_explosive"] == 1, plays["pass_yards"], 0
    )

    # Define stats to aggregate and their aggregation methods
    agg_dict = {
        "play_number": "count",
        "yards_gained": "sum",
        "rush_attempt": "sum",
        "pass_attempt": "sum",
        "dropback": "sum",
        "completion": "sum",
        "rush_yards": "sum",
        "pass_yards": "sum",
        "stuff": "sum",
        "turnover": "sum",
        "TFL": "sum",
        "sack": "sum",
        "success": "sum",
        "explosive": "sum",
        "success_yards": "sum",
        "rush_success": "sum",
        "pass_success": "sum",
        "rush_success_yards": "sum",
        "pass_success_yards": "sum",
        "rush_explosive": "sum",
        "pass_explosive": "sum",
        "rush_explosive_yards": "sum",
        "pass_explosive_yards": "sum",
        "first_down": "sum",
        "second_down": "sum",
        "third_down": "sum",
        "fourth_down": "sum",
        "thirddown_conversion": "sum",
        "fourthdown_conversion": "sum",
        "havoc": "sum",
        "play_duration": "sum",
    }

    # Compute aggregations
    result = (
        plays.groupby(["season", "week", "game_id", "offense", "defense", "home_away"])
        .agg(agg_dict)
        .reset_index()
    )

    # Rename columns
    column_mapping = {
        "play_number": "total_plays",
        "yards_gained": "total_yards",
        "rush_attempt": "rush_attempts",
        "pass_attempt": "pass_attempts",
        "dropback": "dropbacks",
        "completion": "completions",
        "stuff": "stuffs",
        "sack": "sacks",
        "turnover": "turnovers",
        "success": "successes",
        "explosive": "explosives",
        "rush_success": "rush_successes",
        "pass_success": "pass_successes",
        "rush_explosive": "rush_explosives",
        "pass_explosive": "pass_explosives",
        "first_down": "first_downs",
        "second_down": "second_downs",
        "third_down": "third_downs",
        "fourth_down": "fourth_downs",
        "thirddown_conversion": "third_down_conversions",
        "fourthdown_conversion": "fourth_down_conversions",
        "havoc": "havocs",
        "play_duration": "time_of_possession",
    }

    result = result.rename(columns=column_mapping)

    # Fill NaN values
    result = result.fillna(0)

    if verbose:
        end = time.time()
        print(
            f"byplay_to_bygamecum took {int((end - start) // 60)} minutes and {int((end - start) % 60)} seconds to complete."
        )

    return result


# Example usage:
# gameplayscum = byplay_to_gameplayscum(processed_plays)

## By Game - Play Rates

In [7]:
def gameplayscum_to_gameplaysrates(bygame_cum, verbose=False):
    start = time.time()

    try:
        bygame = bygame_cum.copy()

        # List of columns to keep unchanged
        keep_columns = ["season", "week", "offense", "game_id", "defense", "home_away"]

        # Calculate rates and efficiency metrics
        rate_calculations = {
            "ypp": bygame["total_yards"] / bygame["total_plays"],
            "success_rate": bygame["successes"] / bygame["total_plays"],
            "explosive_rate": bygame["explosives"] / bygame["total_plays"],
            "TFL_rate": bygame["TFL"] / bygame["total_plays"],
            "turnover_rate": bygame["turnovers"] / bygame["total_plays"],
            "third_down_conversion_rate": bygame["third_down_conversions"]
            / bygame["third_downs"],
            "fourth_down_conversion_rate": bygame["fourth_down_conversions"]
            / bygame["fourth_downs"],
            "rushpass_split": bygame["rush_attempts"] / bygame["total_plays"],
            "r_ypp": bygame["rush_yards"] / bygame["rush_attempts"],
            "r_success_rate": bygame["rush_successes"] / bygame["rush_attempts"],
            "r_explosive_rate": bygame["rush_explosives"] / bygame["rush_attempts"],
            "r_successyards_per_success": bygame["rush_success_yards"]
            / bygame["rush_successes"],
            "r_explosiveyards_per_explosive": bygame["rush_explosive_yards"]
            / bygame["rush_explosives"],
            "stuff_rate": bygame["stuffs"] / bygame["rush_attempts"],
            "p_ypp": bygame["pass_yards"] / bygame["pass_attempts"],
            "p_success_rate": bygame["pass_successes"] / bygame["pass_attempts"],
            "p_explosive_rate": bygame["pass_explosives"] / bygame["pass_attempts"],
            "p_successyards_per_success": bygame["pass_success_yards"]
            / bygame["pass_successes"],
            "p_explosiveyards_per_explosive": bygame["pass_explosive_yards"]
            / bygame["pass_explosives"],
            "completion_rate": bygame["completions"] / bygame["pass_attempts"],
            "sack_rate": bygame["sacks"] / bygame["dropbacks"],
            "havoc_rate": bygame["havocs"] / bygame["total_plays"],
        }

        # Create DataFrame with calculated rates
        bygamerates = pd.DataFrame(rate_calculations)

        # Add kept columns
        for col in keep_columns:
            if col in bygame.columns:
                bygamerates[col] = bygame[col]

        # Reorder columns
        column_order = keep_columns + list(rate_calculations.keys())
        bygamerates = bygamerates[column_order]

        # Replace infinities and NaNs with 0
        bygamerates = bygamerates.replace([np.inf, -np.inf], np.nan).where(
            pd.notnull(bygamerates), None
        )

        # Sort the DataFrame
        bygamerates = bygamerates.sort_values(
            by=["season", "week", "game_id"]
        ).reset_index(drop=True)

        end = time.time()
        if verbose:
            print(
                f"gameplayscum_to_gameplaysrates took {int((end - start) // 60)} minutes and {int((end - start) % 60)} seconds to complete."
            )

        return bygamerates

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise


# Example usage:
# gameplaysrates = gameplayscum_to_gameplaysrates(gameplayscum)

## By Game - Drives Cumulative

In [8]:
def bydrive_to_gamedrivescum(by_drive, verbose=False):
    start = time.time()

    drives = by_drive.copy()
    # drives = drives[drives['garbage']==0].copy().reset_index(drop=True)

    # Define stats to aggregate and their aggregation methods
    agg_dict = {
        "drive_number": "count",
        "starting_position": "mean",
        "available_yards": "sum",
        "red_zone": "sum",
        "eckel": "sum",
        "drive_success": "sum",
        "score_result": "sum",
        "points_scored": "sum",
        "TD": "sum",
        "FG": "sum",
        "firstdown_drive": "sum",
        "busted_drive": "sum",
        "long_drive": "sum",
        "total_plays": "sum",
        "available_yards_gained": "sum",
        "available_yards_rate": "mean",
        "total_yards": "sum",
        "total_success": "sum",
        "total_explosive": "sum",
        "total_TFL": "sum",
        "havocs": "sum",
        "total_ppa": "sum",
        #'drive_time': 'mean',
        "giveaway": "sum",
        "first_downs": "sum",
        "second_downs": "sum",
        "third_downs": "sum",
        "fourth_down_attempts": "sum",
        "thirddown_conversions": "sum",
        "fourthdown_conversions": "sum",
        "o_penalties": "sum",
        "o_penalty_yards": "sum",
        "d_penalties": "sum",
        "d_penalty_yards": "sum",
    }

    # Compute aggregations
    result = (
        drives.groupby(["season", "week", "game_id", "offense", "defense", "home_away"])
        .agg(agg_dict)
        .reset_index()
    )

    # Rename columns
    column_mapping = {
        "drive_number": "drives",
        "starting_position": "avg_starting_position",
        "red_zone": "red_zone_drives",
        "eckel": "eckel_drives",
        "drive_success": "drive_successes",
        "score_result": "scoring_drives",
        "TD": "TDs",
        "FG": "FGs",
        "firstdown_drive": "firstdown_drives",
        "busted_drive": "busted_drives",
        "long_drive": "long_drives",
        "available_yards_rate": "avg_available_yards_rate",
        "total_success": "total_successes",
        "total_explosive": "total_explosives",
        "total_TFL": "total_TFLs",
        #'drive_time': 'avg_drive_time',
        "giveaway": "giveaways",
    }

    result = result.rename(columns=column_mapping)

    if verbose:
        end = time.time()
        print(
            f"byplay_to_bygamecum took {int((end - start) // 60)} minutes and {int((end - start) % 60)} seconds to complete."
        )

    return result


# Example usage:
# gamedrivescum = bydrive_to_gamedrivescum(by_drive)

## By Game - Drives Rates

In [9]:
def gamedrivescum_to_gamedrivesrates(gamedrivescum: pd.DataFrame, verbose=False):
    start = time.time()

    try:
        bygame = gamedrivescum.copy()

        # List of columns to keep unchanged
        keep_columns = ["season", "week", "game_id", "offense", "defense", "home_away"]

        # Calculate rates and efficiency metrics
        rate_calculations = {
            "ypp": bygame["avg_starting_position"],
            "avg_available_yards": bygame["available_yards"] / bygame["drives"],
            "avg_available_yards_gained": bygame["available_yards_gained"]
            / bygame["drives"],
            "redzone_drive_rate": bygame["red_zone_drives"] / bygame["drives"],
            "eckel_drive_rate": bygame["eckel_drives"] / bygame["drives"],
            "drive_success_rate": bygame["drive_successes"] / bygame["drives"],
            "scoring_drive_rate": bygame["scoring_drives"] / bygame["drives"],
            "pointsscored_per_drive": bygame["points_scored"] / bygame["drives"],
            "pointsscored_per_drive": bygame["points_scored"] / bygame["drives"],
            "TD_drive_rate": bygame["TDs"] / bygame["drives"],
            "FG_drive_rate": bygame["FGs"] / bygame["drives"],
            "firstdown_drive_rate": bygame["firstdown_drives"] / bygame["drives"],
            "busted_drive_rate": bygame["busted_drives"] / bygame["drives"],
            "longdrive_rate": bygame["long_drives"] / bygame["drives"],
            "ppa_per_drive": bygame["total_ppa"] / bygame["drives"],
            "giveaway_drive_rate": bygame["giveaways"] / bygame["drives"],
            "o_penalties_per_drive": bygame["o_penalties"] / bygame["drives"],
            "o_penaltyyards_per_drive": bygame["o_penalty_yards"] / bygame["drives"],
            "d_penalties_per_drive": bygame["d_penalties"] / bygame["drives"],
            "d_penaltyyards_per_drive": bygame["d_penalty_yards"] / bygame["drives"],
        }

        # Create DataFrame with calculated rates
        bygamerates = pd.DataFrame(rate_calculations)

        # Add kept columns
        for col in keep_columns:
            if col in bygame.columns:
                bygamerates[col] = bygame[col]

        # Reorder columns
        column_order = keep_columns + list(rate_calculations.keys())
        bygamerates = bygamerates[column_order]

        # Replace infinities and NaNs with 0
        bygamerates = bygamerates.replace([np.inf, -np.inf], np.nan).where(
            pd.notnull(bygamerates), None
        )

        # Sort the DataFrame
        bygamerates = bygamerates.sort_values(
            by=["season", "week", "game_id"]
        ).reset_index(drop=True)

        end = time.time()
        if verbose:
            print(
                f"gameplayscum_to_gameplaysrates took {int((end - start) // 60)} minutes and {int((end - start) % 60)} seconds to complete."
            )

        return bygamerates

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise


# Example usage:
# gamedrivesrates = gamedrivescum_to_gamedrivesrates(gamedrivescum)

## By Game - All

In [10]:
def merge_playsanddrives(
    gameplaysrates: pd.DataFrame, gamedrivesrates: pd.DataFrame
) -> pd.DataFrame:
    """Merge all drive-level DataFrames and check for consistent length."""
    plays = gameplaysrates.copy()
    drives = gamedrivesrates.copy().drop(columns=["ypp"])

    # Merge plays and drives
    by_game = plays.merge(
        drives,
        on=["season", "week", "game_id", "offense", "defense", "home_away"],
        how="inner",
    )

    # Check if the length of the merged dataframe matches the input dataframes
    if len(by_game) != len(plays) or len(by_game) != len(drives):
        raise ValueError(
            f"Length mismatch after merge: merged length {len(by_game)}, "
            f"plays length {len(plays)}, drives length {len(drives)}, new by_game length {len(by_game)}"
        )

    return by_game

## By Game to By Season

In [ ]:
def bygame_to_byseason(
    game_data: pd.DataFrame, primary_direction: str, verbose: bool = False
) -> pd.DataFrame:
    """
    Aggregate game-level data to season-level data for either offense or defense.

    Args:
        game_data (pd.DataFrame): Game-level data.
        primary_direction (str): Either 'offense' or 'defense'.
        verbose (bool): If True, print execution time.

    Returns:
        pd.DataFrame: Season-level aggregated data.
    """
    import time
    from typing import List

    start = time.time()

    try:
        if primary_direction not in ["offense", "defense"]:
            raise ValueError("primary_direction must be either 'offense' or 'defense'")

        opp_direction = "defense" if primary_direction == "offense" else "offense"

        # Define stat and non-stat columns
        non_stat_columns: List[str] = [
            "season",
            "week",
            "offense",
            "game_id",
            "defense",
            "home_away",
        ]
        stat_columns = [col for col in game_data.columns if col not in non_stat_columns]

        # Create a dictionary for aggregations
        agg_dict = {col: "mean" for col in stat_columns}

        # Perform aggregation
        data = game_data.copy()
        result = data.groupby(["season", primary_direction]).agg(agg_dict)

        # Calculate home and away games
        game_counts = (
            data.groupby(["season", primary_direction, "home_away"])
            .size()
            .unstack(fill_value=0)
        )
        game_counts["games"] = game_counts.get("home", 0) + game_counts.get("away", 0)
        game_counts = (
            game_counts.rename(columns={"home": "home_games", "away": "away_games"})
            if primary_direction == "offense"
            else game_counts.rename(
                columns={"home": "away_games", "away": "home_games"}
            )
        )

        # Merge home and away game counts
        result = result.join(game_counts[["games", "home_games", "away_games"]])

        # Reset index to make 'season' and direction columns
        result = result.reset_index()

        # Reorder columns
        cols = result.columns.tolist()
        cols.remove("games")
        cols.insert(2, "games")
        cols.remove("home_games")
        cols.insert(3, "home_games")
        cols.remove("away_games")
        cols.insert(4, "away_games")
        result = result[cols]
        result = result.rename(columns={primary_direction: "team"})

        # Verbose output
        end = time.time()
        if verbose:
            minutes, seconds = divmod(end - start, 60)
            print(
                f"bygame_to_byseason took {int(minutes)} minutes and {int(seconds)} seconds to complete."
            )

        return result

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

# Rolling Stats and Stat Adjustments

## Rolling Stats

In [ ]:
def calculate_stats_rolling(
    game_data: pd.DataFrame, primary_direction: str, verbose: bool = False
) -> pd.DataFrame:
    """
    Calculate rolling stats for each team in each season and week, using all available data up to each week.

    Args:
        game_data (pd.DataFrame): DataFrame containing game-by-game results
        primary_direction (str): 'offense' or 'defense'
        verbose (bool): If True, print execution time

    Returns:
        pd.DataFrame: DataFrame with calculated rolling stats for each team, season, and week
    """
    start = time.time()

    # Input validation
    if game_data.empty:
        raise ValueError("Input DataFrame is empty")
    if primary_direction not in ["offense", "defense"]:
        raise ValueError("direction must be either 'offense' or 'defense'")
    opp_direction = "defense" if primary_direction == "offense" else "offense"

    data = game_data.copy()

    results: List[pd.DataFrame] = []
    for season in data["season"].unique():
        season_data = data[data["season"] == season].copy().reset_index(drop=True)
        for week in season_data["week"].unique():
            data_up_to_week = (
                season_data[season_data["week"] < week].copy().reset_index(drop=True)
            )
            if not data_up_to_week.empty:
                result = bygame_to_byseason(data_up_to_week, primary_direction)
                result["season"] = season
                result["before_week"] = week

                # Aggregate the list of opponents
                opponents_up_to_week = (
                    data_up_to_week.groupby(primary_direction)[opp_direction]
                    .apply(list)
                    .reset_index()
                    .rename(
                        columns={primary_direction: "team", opp_direction: "opponents"}
                    )
                )

                result = result.merge(opponents_up_to_week, on="team", how="left")
                results.append(result)

    if not results:
        raise ValueError(
            "No data to process. Check if your input DataFrame is empty or if all weeks are filtered out."
        )

    final_result = pd.concat(results, ignore_index=True).reset_index(drop=True)

    # Ensure all expected columns are present
    expected_columns = [
        "season",
        "before_week",
        "team",
        "games",
        "home_games",
        "away_games",
        "opponents",
    ]
    for col in expected_columns:
        if col not in final_result.columns:
            final_result[col] = np.nan

    # Reorder columns
    stat_columns = [col for col in final_result.columns if col not in expected_columns]
    final_result = final_result[expected_columns + stat_columns]

    # Remove numbered columns
    final_result = final_result[
        [col for col in final_result.columns if not isinstance(col, (int, float))]
    ]

    # Rename team column if necessary
    if "team" not in final_result.columns:
        team_column = "offense" if primary_direction == "offense" else "defense"
        final_result = final_result.rename(columns={team_column: "team"})

    # Sort
    final_result = final_result.sort_values(
        by=["season", "team", "before_week"], ascending=[True, True, True]
    ).reset_index(drop=True)

    end = time.time()
    if verbose:
        print(
            f"calculate_rolling_stats took {int((end - start) // 60)} minutes and {int((end - start) % 60)} seconds to complete."
        )

    return final_result

## Adjusting Stats

In [ ]:
def adjust_game_stats(
    game_data: pd.DataFrame,
    primary_direction: str,
    iterations: int,
    verbose: bool = False,
) -> pd.DataFrame:
    """
    Adjust stats for a dataframe of games based on opponent performance, iterating multiple times for better adjustment.

    Parameters:
    game_data (pd.DataFrame): DataFrame containing game-by-game rates
    primary_direction (str): 'offense' or 'defense'
    iterations (int): Number of times to iterate the adjustment process
    verbose (bool): If True, print execution time and progress

    Returns:
    pd.DataFrame: DataFrame with adjusted stats
    """
    # Input validation
    if game_data.empty:
        raise ValueError("Input DataFrame is empty")
    if primary_direction not in ["offense", "defense"]:
        raise ValueError("direction must be either 'offense' or 'defense'")
    if not isinstance(iterations, int) or iterations < 0:
        raise ValueError("iterations must be a non-negative integer")
    opp_direction = "defense" if primary_direction == "offense" else "offense"

    # Get only numeric columns for stats (excluding specific columns)
    exclude_cols = ["season", "week", "offense", "game_id", "defense", "home_away"]
    stat_columns = [col for col in game_data.columns if col not in exclude_cols]

    data = game_data.copy()

    # Return unadjusted data if iterations is 0
    if iterations == 0:
        final_data = data.copy()
        return final_data

    # Initialize results DataFrame
    results = pd.DataFrame()

    seasons = data["season"].unique()
    for season in seasons:
        if verbose:
            print(f"Processing season {season}")

        season_data = data[data["season"] == season].copy().reset_index(drop=True)

        # Set full season data as current
        current_data_o = season_data.copy()
        current_data_d = season_data.copy()

        try:
            # Iterate the adjustment process
            for iteration in range(iterations):
                if verbose:
                    print(f"  Iteration {iteration + 1}/{iterations}")

                # Calculate opponent averages based on current iteration's data
                o_avgs = bygame_to_byseason(current_data_o, "offense").drop(
                    columns=["games", "home_games", "away_games"]
                )
                d_avgs = bygame_to_byseason(current_data_d, "defense").drop(
                    columns=["games", "home_games", "away_games"]
                )

                # Merge game data with opponent averages
                game_and_oppavgs_o = current_data_o.merge(
                    d_avgs,
                    left_on=[
                        "season",
                        "defense",
                    ],  # Fixed: match offense stats with defensive averages
                    right_on=["season", "team"],
                    suffixes=("", "_oppavg"),
                    validate="many_to_one",
                )

                game_and_oppavgs_d = current_data_d.merge(
                    o_avgs,
                    left_on=[
                        "season",
                        "offense",
                    ],  # Fixed: match defense stats with offensive averages
                    right_on=["season", "team"],
                    suffixes=("", "_oppavg"),
                    validate="many_to_one",
                )

                # Calculate adjustments for each stat
                for stat in stat_columns:
                    # For offensive stats, subtract opponent's average
                    game_and_oppavgs_o[stat] = (
                        game_and_oppavgs_o[stat] - game_and_oppavgs_o[f"{stat}_oppavg"]
                    )
                    # For defensive stats, subtract from opponent's average
                    game_and_oppavgs_d[stat] = (
                        game_and_oppavgs_d[f"{stat}_oppavg"] - game_and_oppavgs_d[stat]
                    )

                    # Remove opponent average columns after adjustment
                    game_and_oppavgs_o = game_and_oppavgs_o.drop(
                        columns=[f"{stat}_oppavg"]
                    )
                    game_and_oppavgs_d = game_and_oppavgs_d.drop(
                        columns=[f"{stat}_oppavg"]
                    )

                updated_data_o = game_and_oppavgs_o.drop(columns=["team"])
                updated_data_d = game_and_oppavgs_d.drop(columns=["team"])

                # Prepare for next iteration
                if iteration < iterations - 1:
                    current_data_o = updated_data_o.copy()
                    current_data_d = updated_data_d.copy()
                else:
                    selected_data = (
                        updated_data_o.copy()
                        if primary_direction == "offense"
                        else updated_data_d.copy()
                    )
                    return selected_data

        except Exception as e:
            print(f"Error processing season {season}: {str(e)}")
            raise

In [ ]:
def calculate_adjstats_season(
    game_data: pd.DataFrame,
    primary_direction: str,
    iterations: int = 4,
    verbose: bool = False,
) -> pd.DataFrame:
    """
    Adjust stats for a full season based on opponent performance, iterating multiple times for better adjustment.

    Parameters:
    game_data (pd.DataFrame): DataFrame containing game-by-game rates
    primary_direction (str): 'offense' or 'defense'
    iterations (int): Number of times to iterate the adjustment process
    verbose (bool): If True, print execution time and progress

    Returns:
    pd.DataFrame: DataFrame with adjusted stats
    """

    # Input validation
    if game_data.empty:
        raise ValueError("Input DataFrame is empty")
    if primary_direction not in ["offense", "defense"]:
        raise ValueError("direction must be either 'offense' or 'defense'")
    if not isinstance(iterations, int) or iterations < 0:
        raise ValueError("iterations must be a non-negative integer")
    opp_direction = "defense" if primary_direction == "offense" else "offense"

    data = game_data.copy()
    results = pd.DataFrame()

    for season in data["season"].unique():
        season_data = data[data["season"] == season].copy().reset_index(drop=True)
        adjusted_game_stats = adjust_game_stats(
            season_data, primary_direction, iterations
        )
        adjusted_season_stats = bygame_to_byseason(
            adjusted_game_stats, primary_direction
        )
        results = pd.concat([results, adjusted_season_stats], ignore_index=True)

    final_results = results.sort_values(
        by=["team", "season"], ascending=[True, True]
    ).reset_index(drop=True)

    return final_results

In [ ]:
def calculate_adjstats_rolling(
    game_data: pd.DataFrame,
    primary_direction: str,
    verbose: bool = False,
    iterations: int = 1,
) -> pd.DataFrame:
    """
    Adjust stats for a full season based on opponent performance, iterating multiple times for better adjustment.

    Parameters:
        game_data (pd.DataFrame): DataFrame containing game-by-game rates.
        primary_direction (str): 'offense' or 'defense'.
        iterations (int): Number of times to iterate the adjustment process.
        verbose (bool): If True, print execution time and progress.

    Returns:
        pd.DataFrame: DataFrame with adjusted stats.
    """
    # Input validation
    if game_data.empty:
        raise ValueError("Input DataFrame is empty.")
    if primary_direction not in ["offense", "defense"]:
        raise ValueError("primary_direction must be 'offense' or 'defense'.")
    if not isinstance(iterations, int) or iterations < 0:
        raise ValueError("iterations must be a non-negative integer.")

    opp_direction = "defense" if primary_direction == "offense" else "offense"
    results = []

    # Process each season and week
    for season, season_data in game_data.groupby("season"):
        weeks = sorted(season_data["week"].unique())
        weeks.append(weeks[-1] + 1)  # Add the last week + 1
        for week in weeks:
            data_up_to_week = season_data[season_data["week"] < week]
            if not data_up_to_week.empty:
                # Adjust stats for the season up to the current week
                adjusted_stats = calculate_adjstats_season(
                    data_up_to_week, primary_direction, iterations
                )
                adjusted_stats["before_week"] = week

                # Aggregate opponents faced
                opponents = (
                    data_up_to_week.groupby(primary_direction)[opp_direction]
                    .apply(list)
                    .reset_index()
                    .rename(
                        columns={primary_direction: "team", opp_direction: "opponents"}
                    )
                )

                # Merge adjusted stats with opponents
                adjusted_stats = adjusted_stats.merge(opponents, on="team", how="left")
                results.append(adjusted_stats)

    # Combine all results
    if not results:
        raise ValueError(
            "No data to process. Ensure the DataFrame has valid data and weeks."
        )
    results = pd.concat(results, ignore_index=True)

    # Reorder columns
    reordered_columns = (
        ["team", "season", "before_week"]
        + [
            col
            for col in results.columns
            if col not in ["team", "season", "before_week", "opponents"]
        ]
        + ["opponents"]
    )
    results = results[reordered_columns]

    # Sort final results
    results = results.sort_values(by=["team", "season", "before_week"]).reset_index(
        drop=True
    )

    if verbose:
        print("calculate_adjstats_rolling completed successfully.")

    return results

# Comparisons and Graphing

In [ ]:
def give_me_leaderboard(stat, direction, number, season, combined_adjusted_stats):
    """
    Generate a leaderboard for a specific stat and direction from the combined adjusted stats dataframe.

    Parameters:
    stat (str): The statistic to rank (without _o or _d suffix)
    direction (str): Either 'offense' or 'defense'
    number (int): Number of teams to display in the leaderboard
    season (int): The season to generate the leaderboard for
    combined_adjusted_stats (pd.DataFrame): The combined adjusted stats dataframe
    """
    if direction not in ["offense", "defense"]:
        raise ValueError("direction must be either 'offense' or 'defense'")

    # Determine the correct column name and sorting direction
    stat_column = f"{stat}_o" if direction == "offense" else f"{stat}_d"
    ascending = True if direction == "defense" else False

    # Filter the dataset for the specified season
    season_data = combined_adjusted_stats[combined_adjusted_stats["season"] == season]

    # Sort and rank the teams
    leaderboard = (
        season_data[["team", "games", stat_column]]
        .sort_values(by=stat_column, ascending=ascending)
        .reset_index(drop=True)
    )
    leaderboard[f"{stat}_rank"] = (
        leaderboard[stat_column].rank(ascending=ascending, method="min").astype(int)
    )

    # Rename the stat column for clarity in output
    leaderboard = leaderboard.rename(columns={stat_column: stat})

    # Select top N teams
    leaderboard = leaderboard.head(number)

    print(f"Leaderboard for adjusted {stat} ({direction}) in {season} season:")
    print(leaderboard[["team", "games", stat, f"{stat}_rank"]].to_string(index=False))


# Example usage:
# combined_adjusted_stats = combine_adjusted_stats(bygame_rates)
# give_me_leaderboard('ypp', 'offense', 10, 2023, combined_adjusted_stats)
# give_me_leaderboard('success_rate', 'defense', 5, 2022, combined_adjusted_stats)

In [ ]:
def getImage(id, zoom1=1):
    """
    Load the team logo in full color.

    Parameters:
    id (str): The team identifier for the logo file.

    Returns:
    OffsetImage: The team logo image or a placeholder if not found.
    """
    try:
        img = plt.imread(f"{logo_path}{id}.png")
        return OffsetImage(img, zoom=zoom1)  # Adjust zoom as needed
    except FileNotFoundError:
        print(f"Logo for {id} not found. Returning placeholder.")
        # Return a placeholder image if the logo is not found
        return OffsetImage(np.zeros((1, 1, 4)))  # A 1x1 transparent image


def getGrayImage(id, zoom1=1):
    """
    Load the team logo and convert it to grayscale.

    Parameters:
    id (str): The team identifier for the logo file.

    Returns:
    OffsetImage: The team logo image in grayscale or a placeholder if not found.
    """
    try:
        img = plt.imread(f"{logo_path}{id}.png")
        gray_img = np.dot(
            img[..., :3], [0.2989, 0.5870, 0.1140]
        )  # Grayscale conversion
        return OffsetImage(gray_img, cmap="gray", zoom=zoom1)  # Adjust zoom as needed
    except FileNotFoundError:
        print(f"Logo for {id} not found. Returning placeholder.")
        # Return a placeholder image if the logo is not found
        return OffsetImage(np.zeros((1, 1, 4)))  # A 1x1 transparent image

In [ ]:
def create_graph_comparison(
    data: pd.DataFrame,
    season: int,
    comparison: str,
    statx: str,
    staty: str,
    p5orall: str = "all",
    line: str = "noline",
    adjusted: bool = False,
):
    """
    Create a scatter plot comparing two statistics for a given season in college football, optionally
    filtering by Power 5 (P5) teams and adding diagonal lines or team logos.

    Parameters:
    data (pd.DataFrame): The dataset containing the statistics.
    season (int): The season year to filter data on.
    comparison (str): 'offense' or 'defense' to adjust the titles.
    statx (str): The x-axis statistic column name.
    staty (str): The y-axis statistic column name.
    p5orall (str): 'p5' to filter only Power 5 teams, 'all' for all teams (default: 'all').
    line (str): 'line' to add a diagonal line, 'noline' for no line (default: 'noline').
    """
    try:
        # Offense or defense handling
        suffix = "_o" if comparison == "offense" else "_d"
        suffix_to_drop = "_d" if comparison == "offense" else "_o"
        cols_to_drop = data.filter(like=suffix_to_drop, axis=1).columns

        # Filter data for the specific season
        season_data = data[data["season"] == season]
        season_data = season_data.drop(columns=cols_to_drop)
        season_data = season_data.rename(
            columns=lambda col: col[: -len(suffix)] if col.endswith(suffix) else col
        )

        # Filter for Power 5 teams if specified
        p5 = list(P5schoolsandconferences.keys())
        final_dataset = (
            season_data[season_data["team"].isin(p5)]
            if p5orall == "p5"
            else season_data
        )

        # Error handling for missing stats
        if statx not in final_dataset.columns or staty not in final_dataset.columns:
            raise ValueError(
                f"One or both of the specified statistics ({statx}, {staty}) not found in the data"
            )

        # Get the x and y axis data
        x_axis = final_dataset[statx]
        y_axis = final_dataset[staty]

        # Set axis titles based on the stat dictionary and comparison type
        x_axis_title = (
            f"{comparison[:-1].capitalize()}ive {stats_dictionary.get(statx, 'Unknown Statistic')}"
            if adjusted == False
            else f"{comparison[:-1].capitalize()}ive Adjusted {stats_dictionary.get(statx, 'Unknown Statistic')}"
        )
        y_axis_title = (
            f"{comparison[:-1].capitalize()}ive {stats_dictionary.get(staty, 'Unknown Statistic')}"
            if adjusted == False
            else f"{comparison[:-1].capitalize()}ive Adjusted {stats_dictionary.get(staty, 'Unknown Statistic')}"
        )

        # Create the plot
        fig, ax = plt.subplots(
            figsize=(12, 8)
        )  # Increased figure size for better visibility

        # Calculate the range of the data and add some padding
        x_range = x_axis.max() - x_axis.min()
        y_range = y_axis.max() - y_axis.min()
        padding = 0.1
        x_min = x_axis.min() - (x_range * padding)
        x_max = x_axis.max() + (x_range * padding)
        y_min = y_axis.min() - (y_range * padding)
        y_max = y_axis.max() + (y_range * padding)

        # Set axis limits
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)

        # Plot the scatter points with transparent markers (team logos will replace them)
        ax.scatter(x=x_axis, y=y_axis, alpha=0)

        # Calculate and plot the median lines
        median_x = x_axis.median()
        median_y = y_axis.median()
        ax.axvline(median_x, color="grey", linestyle="--", alpha=0.7)
        ax.axhline(median_y, color="grey", linestyle="--", alpha=0.7)

        # Set plot titles and labels
        ax.set_title(f"{x_axis_title} vs. {y_axis_title} for the {season} CFB Season")
        ax.set_xlabel(x_axis_title)
        ax.set_ylabel(y_axis_title)

        # Optional inversion of axes for defense comparison
        if comparison == "defense":
            ax.invert_xaxis()
            ax.invert_yaxis()

        # Plot a diagonal line if requested
        if line == "line":
            ax.plot(
                [x_min, x_max], [y_min, y_max], color="black", linestyle="-", alpha=0.5
            )

        # Add team logos to the plot (assumes getImage function is defined)
        for x0, y0, team in zip(x_axis, y_axis, final_dataset["team"]):
            try:
                ab = AnnotationBbox(getImage(team), (x0, y0), frameon=False)
                ax.add_artist(ab)
            except FileNotFoundError:
                print(f"Logo for {team} not found")

        # Add gridlines for better readability
        ax.grid(True, linestyle=":", alpha=0.6)

        # Adjust layout to prevent cutting off labels
        plt.tight_layout()

        # Display the plot
        plt.show()

    except ValueError as ve:
        print(f"ValueError: {str(ve)}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")


# Usage example:
# create_graph_comparison(data=df, season=2023, comparison='offense', statx='adj_epa_per_play', staty='adj_success_rate')

In [ ]:
def create_weekly_performance_graph(
    data: pd.DataFrame,
    season: int,
    statx: str,
    staty: str,
    comparison: str,
    highlight_team: str = None,
    adjusted: bool = False,
):
    """
    Create a scatter plot showing weekly performance for two statistics with team logos and week numbers.

    Parameters:
    data (pd.DataFrame): The dataset containing weekly statistics
    season (int): The season year to filter data on
    statx (str): The x-axis statistic column name
    staty (str): The y-axis statistic column name
    comparison (str): 'offense' or 'defense' to adjust the titles
    highlight_team (str): Optional team name to highlight (others will be grayed out)
    adjusted (bool): Whether to use adjusted statistics
    """
    try:
        # Offense or defense handling
        suffix = "_o" if comparison == "offense" else "_d"
        suffix_to_drop = "_d" if comparison == "offense" else "_o"
        cols_to_drop = data.filter(like=suffix_to_drop, axis=1).columns

        # Filter data for the specific season
        season_data = data[data["season"] == season].copy()
        season_data = season_data.drop(columns=cols_to_drop)
        season_data = season_data.rename(
            columns=lambda col: col[: -len(suffix)] if col.endswith(suffix) else col
        )

        # Create the plot
        fig, ax = plt.subplots(figsize=(15, 10))

        # Calculate overall ranges for axis limits
        x_range = season_data[statx].max() - season_data[statx].min()
        y_range = season_data[staty].max() - season_data[staty].min()
        padding = 0.1
        x_min = season_data[statx].min() - (x_range * padding)
        x_max = season_data[statx].max() + (x_range * padding)
        y_min = season_data[staty].min() - (y_range * padding)
        y_max = season_data[staty].max() + (y_range * padding)

        # Set axis limits
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)

        # Sort teams so highlighted team is plotted last
        teams = list(season_data["team"].unique())
        if highlight_team and highlight_team in teams:
            teams.remove(highlight_team)
            teams.append(highlight_team)

        # Plot team data
        for team in teams:
            team_data = season_data[season_data["team"] == team].sort_values("week")

            # Determine style based on highlight_team
            is_highlighted = team == highlight_team or highlight_team is None
            line_alpha = 1.0 if is_highlighted else 0.15
            logo_alpha = 1.0 if is_highlighted else 0.15
            text_alpha = 1.0 if is_highlighted else 0.15
            line_color = "red" if team == highlight_team else "gray"
            zorder = 2 if is_highlighted else 1

            # Connect points with lines if they're consecutive weeks
            if len(team_data) > 1:
                ax.plot(
                    team_data[statx],
                    team_data[staty],
                    alpha=line_alpha,
                    linestyle="--",
                    color=line_color,
                    linewidth=2 if is_highlighted else 1,
                    zorder=zorder,
                )

            # Add logos with week numbers
            for _, row in team_data.iterrows():
                try:
                    # print(f"Loading logo for team: {team}")
                    logo = (
                        getImage(team)
                        if team == highlight_team
                        else getGrayImage(team, 0.75)
                    )

                    ab = AnnotationBbox(
                        logo,
                        (row[statx], row[staty]),
                        frameon=False,
                        zorder=zorder + 1,
                        alpha=logo_alpha,
                        box_alignment=(0.5, 0.5),
                    )
                    ax.add_artist(ab)

                    ax.annotate(
                        f"G{int(row['games'])}",
                        xy=(row[statx], row[staty]),
                        xytext=(0, -14),
                        textcoords="offset points",
                        ha="center",
                        va="top",
                        alpha=text_alpha,
                        fontsize=8,
                        color=line_color if is_highlighted else "gray",
                        weight="bold" if is_highlighted else "normal",
                        zorder=zorder + 1,
                    )

                except Exception as e:
                    print(f"An error occurred while processing {team}: {str(e)}")
                except FileNotFoundError:
                    print(f"Logo for {team} not found")
                    # Fallback to a simple scatter point if logo isn't found
                    ax.scatter(
                        row[statx],
                        row[staty],
                        alpha=logo_alpha,
                        color=line_color,
                        zorder=zorder,
                    )

        # Calculate and plot the median lines
        median_x = season_data[statx].median()
        median_y = season_data[staty].median()
        ax.axvline(median_x, color="grey", linestyle="--", alpha=0.7, zorder=0)
        ax.axhline(median_y, color="grey", linestyle="--", alpha=0.7, zorder=0)

        # Set plot titles and labels
        x_axis_title = f"{comparison.capitalize()}ive {stats_dictionary.get(statx, 'Unknown Statistic')}"
        y_axis_title = f"{comparison.capitalize()}ive {stats_dictionary.get(staty, 'Unknown Statistic')}"
        if adjusted:
            x_axis_title = f"Adjusted {x_axis_title}"
            y_axis_title = f"Adjusted {y_axis_title}"

        ax.set_title(f"{x_axis_title} vs. {y_axis_title} by Week - {season} Season")
        ax.set_xlabel(x_axis_title)
        ax.set_ylabel(y_axis_title)

        # Optional inversion of axes for defense comparison
        if comparison == "defense":
            ax.invert_xaxis()
            ax.invert_yaxis()

        # Add gridlines
        ax.grid(True, linestyle=":", alpha=0.6, zorder=0)

        # Adjust layout
        plt.tight_layout()

        # Display the plot
        plt.show()

    except ValueError as ve:
        print(f"ValueError: {str(ve)}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")


# Usage example:
# create_weekly_performance_graph(
#     data=df,
#     season=2023,
#     statx='yards_per_play',
#     staty='success_rate',
#     comparison='offense',
#     highlight_team='Alabama'
# )

In [ ]:
# Set plot titles and labels
#        x_axis_title = f"{comparison[:-1].capitalize()}ive {stats_dictionary.get(statx, 'Unknown Statistic')}"
#        y_axis_title = f"{comparison[:-1].capitalize()}ive {stats_dictionary.get(staty, 'Unknown Statistic')}"